In [29]:
import os
import subprocess
import math
from pathlib import Path
import shutil
import numpy as np
import random
import copy
from scipy.stats import pearsonr
from pprint import pprint
from argparse import ArgumentParser
import pandas as pd
from sklearn.model_selection import train_test_split

In [38]:
# --- Configuration ---
RDIR = "/home/luis_ubuntu/unixdir/Peptide_Binding/smm_comparison/"
DDIR = "/home/luis_ubuntu/unixdir/Peptide_Binding/Data"

#alleles = [
#    "A0101", "A0202", "A2402", "A3001", "A3101", "A6901",
#    "B0801", "B3501", "B4403", "B5801", "A0201", "A2301",
#    "A2403", "A3002", "A6801", "B0702", "B2705", "B4402", "B5701"
#]

alleles = [
    "B5801", "A0201", "A3101"
]

lambdas = [0.02, 0.08, 0.1]
epis = [0.01, 0.04, 0.08]
folds = range(4)

In [34]:
# --- Pearson correlation calculator ---
def pearson_from_pairs(pairs):
    n = len(pairs)
    if n == 0:
        return 0.0, float("inf")
    
    x = [p[0] for p in pairs]
    y = [p[1] for p in pairs]
    
    x0 = sum(x) / n
    y0 = sum(y) / n
    
    t = nx = ny = err = 0.0
    for i in range(n):
        dx = x[i] - x0
        dy = y[i] - y0
        t += dx * dy
        nx += dx * dx
        ny += dy * dy
        err += (x[i] - y[i]) ** 2
    
    if nx * ny == 0:
        pcc = 0.0
    else:
        pcc = t / math.sqrt(nx * ny)
    
    mse = err / n
    return pcc, mse

def make_and_enter(dir_path):
    path = Path(dir_path)
    path.mkdir(exist_ok=True)
    os.chdir(path)

def run_training_and_evaluation(RDIR, train_file, eval_file, mat_file, pred_file, _lambda, _epsilon):
    # Run training
    if not Path(mat_file).exists():
        with open(mat_file, "w") as fout:
            subprocess.run(
                [
                    "python", f"{RDIR}/smm_gradient_descent.py",
                    "-l", str(_lambda),
                    "-epi", str(_epsilon),
                    "-t", train_file
                ],
                stdout=fout,
                stderr=subprocess.DEVNULL,
                env={**os.environ, "QT_QPA_PLATFORM": "offscreen"}
            )

    # Run evaluation
    if not Path(pred_file).exists():
        with open(pred_file, "w") as fout:
            subprocess.run(
                [
                    "python", f"{RDIR}/pep2score.py",
                    "-mat", mat_file,
                    "-f", eval_file
                ],
                stdout=fout,
                stderr=subprocess.DEVNULL,
                env={**os.environ, "QT_QPA_PLATFORM": "offscreen"}
            )

def save_final_predictions(allele, best_lambda, best_epsilon):
    """
    Concatenates all .pred files from the best hyperparameter directory
    and writes the result into {allele}_final_predictions file.
    """
    pred_lines = []
    best_pred_dir = Path(f"l.{best_lambda}/epi.{best_epsilon}")

    # Collect lines from all .pred files, ignoring comments
    for pred_file in best_pred_dir.glob("*.pred"):
        with open(pred_file) as pf:
            for line in pf:
                if not line.startswith("#") and line.strip():
                    pred_lines.append(line)

    # Write to the output file in the parent directory
    output_file = f"{RDIR}/{allele}.res/{allele}_prediction"
    with open(output_file, "w") as final_out:
        final_out.writelines(pred_lines)

LOOP FOR UN-NESTED

In [35]:
os.chdir(RDIR)
# --- Main loop ---
for allele in alleles:
    best_pcc = -1000
    best_model = ""
    best_lambda = ""
    best_epsilon = ""

    make_and_enter(f"{allele}_unnested.res")

    for l in lambdas:
        make_and_enter(f"l.{l}")

        for epi in epis:
            make_and_enter(f"epi.{epi}")

            preds = []

            for n in folds:
                mat_file = f"mat.{n}"
                pred_file = f"c00{n}.pred"
                
                train_file = f"{DDIR}/{allele}/f00{n}"
                eval_file = f"{DDIR}/{allele}/c00{n}"

                # Run training and evalutation
                run_training_and_evaluation(RDIR, train_file, eval_file, mat_file=mat_file, pred_file=pred_file, _lambda=l, _epsilon=epi)

                # Parse predictions
                with open(pred_file) as pf:
                    for line in pf:
                        if "#" not in line and line.strip():
                            try:
                                parts = line.strip().split()
                                preds.append((float(parts[1]), float(parts[2])))
                            except:
                                continue

            # Compute PCC and MSE
            pcc, mse = pearson_from_pairs(preds)
            eval_output = f"{allele} lambda {l} epsilon {epi} PCC {pcc:.5f} MSE {mse:.5f}"
            print(eval_output)

            if pcc > best_pcc:
                best_pcc = pcc
                best_lambda = l
                best_epsilon = epi
                best_model = f"lambda {best_lambda} epsilon {best_epsilon}"

            os.chdir("..")  # up from epi.X

        os.chdir("..")  # up from l.X
    
    # Final result
    print("\nBest model for allele", allele,": ", best_model, "with correlation", f"{best_pcc:.5f}\n")

    # save_final_predictions(allele, best_lambda, best_epsilon)

    #pairs = []
    #with open(f"{allele}_prediction") as f:
    #    for line in f:
    #        parts = line.strip().split()
    #        try:
    #            x = float(parts[1])  # second column
    #            y = float(parts[2])  # third column
    #            pairs.append((x, y))
    #        except (IndexError, ValueError):
    #            continue
                
    #pcc, mse = pearson_from_pairs(pairs)
    #eval_output = f"Final prediciton for allele:{allele}; PCC {pcc:.5f} MSE {mse:.5f}"
    #print(eval_output)

    eval_file = f"{DDIR}/{allele}/c004"
    mat_files = [f"{RDIR}/{allele}_unnested.res/l.{best_lambda}/epi.{best_epsilon}/mat.{i}" for i in range(4)]
    pred_files = [f"validation_{i}.pred" for i in range(4)]
    final_pred_file = "validation_avg.pred"

    for i in range(4):
        mat_file = mat_files[i]
        pred_file = pred_files[i]
    
        # Run evaluation
        if not Path(pred_file).exists():
            with open(pred_file, "w") as fout:
                subprocess.run(
                    [
                        "python", f"{RDIR}/pep2score.py",
                        "-mat", mat_file,
                        "-f", eval_file
                    ],
                    stdout=fout,
                    stderr=subprocess.DEVNULL,
                    env={**os.environ, "QT_QPA_PLATFORM": "offscreen"}
                )
    
    # Step 2: Average predictions
    # Assumes each pred_file has one score per line
    all_preds = []
    all_peptide_names = []
    all_real_values = []
    
    # Read all prediction files and collect data
    for idx, pred_file in enumerate(pred_files):
        with open(pred_file) as f:
            preds = []
            peptide_names = []
            real_values = []
            if idx == 0:
                # On first file, also store peptide names and real values
                for line in f:
                    if line.strip() and not line.strip().startswith('#'):
                        parts = line.strip().split()
                        peptide_names.append(parts[0])
                        real_values.append(float(parts[2]))
                        preds.append(float(parts[1]))     
            else:
                for line in f:
                    if line.strip() and not line.strip().startswith('#'):
                        parts = line.strip().split()
                        preds.append(float(parts[1]))  # Or which column is prediction?
    
            all_preds.append(preds)
            
    all_peptide_names.append(peptide_names)
    all_real_values.append(real_values)
    
    # Convert to numpy array for row-wise mean
    all_preds = np.array(all_preds)  # shape: (4, N)
    mean_preds = np.mean(all_preds, axis=0)
    
    # Save final averaged predictions
    with open(final_pred_file, "w") as f:
        for name, pred, real in zip(peptide_names, mean_preds, real_values):
            f.write(f"{name}\t{pred:.6f}\t{real:.6f}\n")
    
    print(f"Saved averaged predictions to {final_pred_file}")

    os.chdir("..")  # up from allele.res

B5801 lambda 0.02 epsilon 0.01 PCC 0.68702 MSE 0.01938
B5801 lambda 0.02 epsilon 0.04 PCC 0.67719 MSE 0.02012
B5801 lambda 0.02 epsilon 0.08 PCC 0.66210 MSE 0.02320
B5801 lambda 0.08 epsilon 0.01 PCC 0.68705 MSE 0.01937
B5801 lambda 0.08 epsilon 0.04 PCC 0.67719 MSE 0.02012
B5801 lambda 0.08 epsilon 0.08 PCC 0.66212 MSE 0.02320
B5801 lambda 0.1 epsilon 0.01 PCC 0.68704 MSE 0.01937
B5801 lambda 0.1 epsilon 0.04 PCC 0.67724 MSE 0.02011
B5801 lambda 0.1 epsilon 0.08 PCC 0.66212 MSE 0.02319

Best model for allele B5801 :  lambda 0.08 epsilon 0.01 with correlation 0.68705

Saved averaged predictions to validation_avg.pred


LOOP FOR NESTED

In [39]:
# --- Pearson correlation calculator ---
def pearson_from_pairs(pairs):
    n = len(pairs)
    if n == 0:
        return 0.0, float("inf")
    
    x = [p[0] for p in pairs]
    y = [p[1] for p in pairs]
    
    x0 = sum(x) / n
    y0 = sum(y) / n
    
    t = nx = ny = err = 0.0
    for i in range(n):
        dx = x[i] - x0
        dy = y[i] - y0
        t += dx * dy
        nx += dx * dx
        ny += dy * dy
        err += (x[i] - y[i]) ** 2
    
    if nx * ny == 0:
        pcc = 0.0
    else:
        pcc = t / math.sqrt(nx * ny)
    
    mse = err / n
    return pcc, mse

def make_and_enter(dir_path):
    path = Path(dir_path)
    path.mkdir(exist_ok=True)
    os.chdir(path)

def concat_train_files(n_list, allele, DDIR, held_out_fold):
    output_file = Path(f"conc_f00{held_out_fold}")
    if output_file.exists():
        return output_file  # Return existing file without rewriting
    
    with open(output_file, "w") as outfile:
        for n in n_list:
            file_path = Path(f"{DDIR}/{allele}/c00{n}")
            with open(file_path) as infile:
                outfile.writelines(infile.readlines())

def run_training_and_evaluation(RDIR, train_file, eval_file, mat_file, pred_file, _lambda, _epsilon):
    # Run training
    if not Path(mat_file).exists():
        with open(mat_file, "w") as fout:
            subprocess.run(
                [
                    "python", f"{RDIR}/smm_gradient_descent.py",
                    "-l", str(_lambda),
                    "-epi", str(_epsilon),
                    "-t", train_file
                ],
                stdout=fout,
                stderr=subprocess.DEVNULL,
                env={**os.environ, "QT_QPA_PLATFORM": "offscreen"}
            )

    # Run evaluation
    if not Path(pred_file).exists():
        with open(pred_file, "w") as fout:
            subprocess.run(
                [
                    "python", f"{RDIR}/pep2score.py",
                    "-mat", mat_file,
                    "-f", eval_file
                ],
                stdout=fout,
                stderr=subprocess.DEVNULL,
                env={**os.environ, "QT_QPA_PLATFORM": "offscreen"}
            )

def collect_outer_preds(allele):
    preds = []
    preds_for_file = []
    for path in Path('.').rglob('*.outer_pred'):
        with open(path) as pf:
            for line in pf:
                if "#" not in line and line.strip():
                    try:
                        parts = line.strip().split()
                        preds_for_file.append(f"{parts[0]} {parts[1]} {parts[2]}\n")
                    except:
                        continue

    # Save all collected predictions to a single file
    output_file = Path(f"{allele}_prediction")
    with open(output_file, "w") as out:
        out.writelines(preds_for_file)
    
    print(f"Saved {len(preds_for_file)} predictions to {output_file}")

In [47]:
os.chdir(RDIR)
# --- Loop over all alleles ---
for allele in alleles:
    make_and_enter(f"{allele}_nested.res")

    # --- Outer Validation loop ---
    for n in folds:
        inner_loop_files = [0, 1, 2, 3]
        inner_loop_files.remove(n)
    
        make_and_enter(f"validation_set_{n}")

        # copy the validation files and create the new concatenated training files
        for m in inner_loop_files:
            # Copy the evaluation file
            eval_file = f"{DDIR}/{allele}/c00{m}"
            shutil.copy(eval_file, f"c00{m}")
            # Create the concatenated trining files
            train_files = inner_loop_files.copy()
            train_files.remove(m)

            concat_train_files(train_files, allele, DDIR, m)
        
        best_pcc = -1000
        best_model = ""
        best_lambda = ""
        best_epsilon = ""
        
    # --- Hyperparameters loop ---    
        for l in lambdas:
            make_and_enter(f"l.{l}")
                
            for epi in epis:
                make_and_enter(f"epi.{epi}")
    
                preds = []

                # --- Inner CV loop ---
                for m in inner_loop_files:
                    
                    # Define the files to start running
                    mat_file = f"mat.{m}"
                    pred_file = f"c00{m}.pred"
                    
                    #eval_file = f"../../c00{m}"
                    eval_file = f"{RDIR}{allele}_nested.res/validation_set_{n}/c00{m}"
                    #train_file = f"../../conc_f00{m}"
                    train_file = f"{RDIR}{allele}_nested.res/validation_set_{n}/conc_f00{m}"
                    
                    # Run training and evalutation
                    
                    run_training_and_evaluation(RDIR, train_file, eval_file, mat_file=mat_file, pred_file=pred_file, _lambda=l, _epsilon=epi)
                    
                    with open(pred_file) as pf:
                        for line in pf:
                            if "#" not in line and line.strip():
                                try:
                                    parts = line.strip().split()
                                    preds.append((float(parts[1]), float(parts[2])))
                                except:
                                    continue
                
                # Compute PCC and MSE
                pcc, mse = pearson_from_pairs(preds)
                eval_output = f"{allele}; Outer validation set = {n}, lambda = {l} epsilon = {epi} PCC {pcc:.5f} MSE {mse:.5f}"
                print(eval_output)

                if pcc > best_pcc:
                    best_pcc = pcc
                    best_lambda = l
                    best_epsilon = epi
                    best_model = f"lambda {best_lambda} epsilon {best_epsilon}"
                    
                
                # Leave epi.{epi}
                os.chdir("..")
        
            # Leave l.{l}
            os.chdir("..")

        # Final result
        print("\nBest model for allele", allele,"and Outer validation set '", n, "', :", best_model, "with correlation", f"{best_pcc:.5f}\n")

        # Copy the final evaluation file
        final_eval_file = f"{DDIR}/{allele}/c00{n}"
        shutil.copy(final_eval_file, f"final_evaluation_c00{n}")
        
        # Copy the full training file to use when the best HP are chosen
        full_train_file = f"{DDIR}/{allele}/f00{n}"
        shutil.copy(full_train_file, f"final_training_f00{n}")

        # Define the files to start running
        mat_file = f"mat.{n}"
        pred_file = f"c00{n}.outer_pred"
                    
        eval_file = f"final_evaluation_c00{n}"
        train_file = f"final_training_f00{n}"

        # Run training and evaluation
        run_training_and_evaluation(RDIR, train_file, eval_file, mat_file=mat_file, pred_file=pred_file, _lambda=best_lambda, _epsilon=best_epsilon)
        
        # Leave validation_set_{n}
        os.chdir("..")

    # Concatenate predictions for allele and get the final measure
    #collect_outer_preds(allele)

    #pairs = []
    #with open(f"{allele}_prediction") as f:
    #    for line in f:
    #        parts = line.strip().split()
    #        try:
    #            x = float(parts[1])  # second column
    #            y = float(parts[2])  # third column
    #            pairs.append((x, y))
    #        except (IndexError, ValueError):
    #            continue
                
    #pcc, mse = pearson_from_pairs(pairs)
    #eval_output = f"Final prediciton for allele:{allele}; PCC {pcc:.5f} MSE {mse:.5f}"
    #print(eval_output)
    eval_file = f"{DDIR}/{allele}/c004"
    mat_files = [f"{RDIR}/{allele}_nested.res/validation_set_{i}/mat.{i}" for i in range(4)]
    pred_files = [f"validation_{i}.pred" for i in range(4)]
    final_pred_file = "validation_avg.pred"

    for i in range(4):
        mat_file = mat_files[i]
        pred_file = pred_files[i]
    
        # Run evaluation
        if not Path(pred_file).exists():
            with open(pred_file, "w") as fout:
                subprocess.run(
                    [
                        "python", f"{RDIR}/pep2score.py",
                        "-mat", mat_file,
                        "-f", eval_file
                    ],
                    stdout=fout,
                    stderr=subprocess.DEVNULL,
                    env={**os.environ, "QT_QPA_PLATFORM": "offscreen"}
                )
    
    # Step 2: Average predictions
    # Assumes each pred_file has one score per line
    all_preds = []
    peptide_names = []
    real_values = []
    
    # Read all prediction files and collect data
    for idx, pred_file in enumerate(pred_files):
        with open(pred_file) as f:
            preds = []
            if idx == 0:
                # On first file, also store peptide names and real values
                for line in f:
                    if line.strip() and not line.strip().startswith('#'):
                        parts = line.strip().split()
                        peptide_names.append(parts[0])
                        real_values.append(float(parts[2]))
                        preds.append(float(parts[1]))     
            else:
                for line in f:
                    if line.strip() and not line.strip().startswith('#'):
                        parts = line.strip().split()
                        preds.append(float(parts[1])) 
    
            all_preds.append(preds)

    
    # Convert to numpy array for row-wise mean
    all_preds = np.array(all_preds)  # shape: (4, N)
    mean_preds = np.mean(all_preds, axis=0)
    
    # Save final averaged predictions
    with open(final_pred_file, "w") as f:
        for name, pred, real in zip(peptide_names, mean_preds, real_values):
            f.write(f"{name}\t{pred:.6f}\t{real:.6f}\n")
    
    print(f"Saved averaged predictions to {final_pred_file}")

    # Leave {allele}.res
    os.chdir("..")

B5801; Outer validation set = 0, lambda = 0.02 epsilon = 0.01 PCC 0.63031 MSE 0.02553
B5801; Outer validation set = 0, lambda = 0.02 epsilon = 0.04 PCC 0.60673 MSE 0.02789
B5801; Outer validation set = 0, lambda = 0.02 epsilon = 0.08 PCC 0.57982 MSE 0.03046
B5801; Outer validation set = 0, lambda = 0.08 epsilon = 0.01 PCC 0.63041 MSE 0.02551
B5801; Outer validation set = 0, lambda = 0.08 epsilon = 0.04 PCC 0.60685 MSE 0.02786
B5801; Outer validation set = 0, lambda = 0.08 epsilon = 0.08 PCC 0.57988 MSE 0.03044
B5801; Outer validation set = 0, lambda = 0.1 epsilon = 0.01 PCC 0.63043 MSE 0.02551
B5801; Outer validation set = 0, lambda = 0.1 epsilon = 0.04 PCC 0.60692 MSE 0.02785
B5801; Outer validation set = 0, lambda = 0.1 epsilon = 0.08 PCC 0.57991 MSE 0.03043

Best model for allele B5801 and Outer validation set ' 0 ', : lambda 0.1 epsilon 0.01 with correlation 0.63043

B5801; Outer validation set = 1, lambda = 0.02 epsilon = 0.01 PCC 0.64165 MSE 0.02384
B5801; Outer validation set = 